In [1]:
import logging
from pathlib import Path
from typing import Dict, List, Optional

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *

class ParquetReader:
    def __init__(self, spark_config: Dict[str, str] = None):
        """
        Khởi tạo ParquetReader với cấu hình Spark tùy chỉnh
        """
        self.logger = self._setup_logging()
        self.spark = self._create_spark_session(spark_config)
        
    def _setup_logging(self) -> logging.Logger:
        """Cấu hình logging"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        return logging.getLogger(__name__)

    def _create_spark_session(self, spark_config: Dict[str, str] = None) -> SparkSession:
        """Tạo SparkSession với cấu hình tùy chỉnh"""
        default_config = {
            "spark.sql.files.maxPartitionBytes": "128MB",
            "spark.sql.shuffle.partitions": "10",
            "spark.driver.memory": "2g",
            "spark.executor.memory": "2g"
        }
        
        builder = SparkSession.builder.appName("ParquetReader")
        
        # Áp dụng cấu hình mặc định
        for key, value in default_config.items():
            builder = builder.config(key, value)
            
        # Ghi đè bằng cấu hình tùy chỉnh nếu có
        if spark_config:
            for key, value in spark_config.items():
                builder = builder.config(key, value)
                
        return builder.getOrCreate()

    def read_parquet(
        self, 
        input_path: str,
        columns: List[str] = None,
        filters: List[str] = None,
        partition_filter: Dict[str, str] = None
    ) -> DataFrame:
        """
        Đọc dữ liệu từ file/thư mục Parquet
        
        Args:
            input_path: Đường dẫn đến file/thư mục Parquet
            columns: Danh sách các cột cần đọc
            filters: Các điều kiện lọc dạng SQL
            partition_filter: Filter theo partition columns
        """
        try:
            self.logger.info(f"Đọc dữ liệu từ: {input_path}")
            
            # Kiểm tra path tồn tại
            if not Path(input_path).exists():
                raise ValueError(f"Path không tồn tại: {input_path}")
            
            # Đọc dữ liệu
            df = self.spark.read.option("mergeSchema", "true").parquet(input_path)
            
            # Select các cột cần thiết
            if columns:
                df = df.select(columns)
            
            # Áp dụng partition filter nếu có
            if partition_filter:
                for column_name, value in partition_filter.items():
                    df = df.filter(col(column_name) == value)  # Đổi tên biến để tránh lỗi

            
            # Áp dụng các điều kiện lọc
            if filters:
                for filter_expr in filters:
                    df = df.filter(expr(filter_expr))  # Sử dụng expr() function
            
            # Log thông tin
            self.logger.info(f"Schema của DataFrame:")
            df.printSchema()
            
            count = df.count()
            self.logger.info(f"Số lượng records: {count}")
            
            return df
            
        except Exception as e:
            self.logger.error(f"Lỗi khi đọc Parquet: {str(e)}", exc_info=True)
            raise

    def analyze_data(self, df: DataFrame) -> Dict:
        """Phân tích cơ bản về DataFrame"""
        try:
            stats = {}
            
            # Thống kê cơ bản
            stats['record_count'] = df.count()
            stats['column_count'] = len(df.columns)
            
            # Đếm null values cho mỗi cột
            null_counts = {}
            for column in df.columns:
                null_count = df.filter(col(column).isNull()).count()
                null_counts[column] = null_count
            stats['null_counts'] = null_counts
            
            # Thống kê cho các cột số
            numeric_stats = df.describe().collect()
            stats['numeric_stats'] = {row['summary']: {c: row[c] for c in df.columns if c != 'summary'}
                                    for row in numeric_stats}
            
            return stats
            
        except Exception as e:
            self.logger.error(f"Lỗi khi phân tích dữ liệu: {str(e)}", exc_info=True)
            raise

    def cleanup(self):
        """Dọn dẹp tài nguyên"""
        if self.spark:
            self.spark.stop()

def main():
    # Cấu hình Spark
    spark_config = {
        "spark.driver.memory": "2g",
        "spark.executor.memory": "2g"
    }
    
    reader = ParquetReader(spark_config)
    
    try:
        # Đọc dữ liệu với filters
        df = reader.read_parquet(
            input_path="D:/Project/delta_lake/bronze/yelp_data/",
            columns=["business_id", "name", "city", "state", "stars"],
            filters=["stars >= 4.0"],  # Sử dụng string expression
            partition_filter={"state": "AZ"}
        )
        
        # Show một số rows
        print("\nMẫu dữ liệu:")
        df.show(5)
        
        # Phân tích dữ liệu
        stats = reader.analyze_data(df)
        
        print("\nThống kê:")
        print(f"Tổng số records: {stats['record_count']}")
        print(f"Số cột: {stats['column_count']}")
        
        print("\nSố lượng giá trị null theo cột:")
        for col, count in stats['null_counts'].items():
            print(f"{col}: {count}")
            
        print("\nThống kê cho các cột số:")
        for metric, values in stats['numeric_stats'].items():
            print(f"\n{metric}:")
            for col, val in values.items():
                print(f"{col}: {val}")
        
    except Exception as e:
        reader.logger.error(f"Lỗi trong main: {str(e)}", exc_info=True)
    finally:
        reader.cleanup()

if __name__ == "__main__":
    main()

2025-01-16 14:06:41,482 - __main__ - INFO - Đọc dữ liệu từ: D:/Project/delta_lake/bronze/yelp_data/
2025-01-16 14:06:49,562 - __main__ - INFO - Schema của DataFrame:


root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- stars: double (nullable = true)



2025-01-16 14:06:55,072 - __main__ - INFO - Số lượng records: 4848



Mẫu dữ liệu:
+--------------------+--------------------+------+-----+-----+
|         business_id|                name|  city|state|stars|
+--------------------+--------------------+------+-----+-----+
|E29l7NtSA3xu_i9zL...|      Foster's Shoes|Tucson|   AZ|  4.0|
|PcZ00r97wYVYfvzpc...|       Garage on 4th|Tucson|   AZ|  4.0|
|1H7tQr5ntTm2Gm9LD...|        Mr Baja Fish|Tucson|   AZ|  4.5|
|5yDc6oeynmMLlTz80...| Melvin M Dixon, DDS|Tucson|   AZ|  4.0|
|fFZWERbPXYGlRC9db...|Justin's Diamond ...|Tucson|   AZ|  4.0|
+--------------------+--------------------+------+-----+-----+
only showing top 5 rows


Thống kê:
Tổng số records: 4848
Số cột: 5

Số lượng giá trị null theo cột:
business_id: 0
name: 0
city: 0
state: 0
stars: 0

Thống kê cho các cột số:

count:
business_id: 4848
name: 4848
city: 4848
state: 4848
stars: 4848

mean:
business_id: None
name: 1702.0
city: None
state: None
stars: 4.414707095709571

stddev:
business_id: None
name: None
city: None
state: None
stars: 0.390595389659072

In [4]:
from pyspark.sql import SparkSession
from delta import *
import os

# Chuyển đổi đường dẫn sang dạng chuẩn
csv_path = os.path.abspath("D:/amazon_products_20241211_031454.csv")
delta_path = os.path.abspath("D:/Project/Test")

# Khởi tạo Spark Session
builder = SparkSession.builder \
    .appName("CSV to Delta Lake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:3.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Thêm try-except để xem lỗi chi tiết
try:
    # Đọc file CSV
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv(csv_path)

    print("Đã đọc file CSV thành công")
    
    # Hiển thị schema
    df.printSchema()

    # Lưu vào Delta Lake
    df.write \
        .format("delta") \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .save(delta_path)

    print("Đã lưu vào Delta Lake thành công")

    # Đọc lại để kiểm tra
    delta_df = spark.read.format("delta").load(delta_path)
    delta_df.printSchema()
    delta_df.show()

except Exception as e:
    print("Lỗi xảy ra:", str(e))
    
finally:
    # Đóng Spark Session
    spark.stop()

Đã đọc file CSV thành công
root
 |-- title: string (nullable = true)
 |-- price: string (nullable = true)
 |-- old_price: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- purchases: string (nullable = true)
 |-- detail_style: string (nullable = true)
 |-- detail_brand: string (nullable = true)
 |-- detail_model_name: string (nullable = true)
 |-- detail_memory_storage_capacity: string (nullable = true)
 |-- detail_screen_size: string (nullable = true)
 |-- detail_display_resolution_maximum: string (nullable = true)
 |-- detail_description: string (nullable = true)
 |-- detail_digital_storage_capacity: string (nullable = true)
 |-- detail_hard_disk_interface: string (nullable = true)
 |-- detail_connectivity_technology: string (nullable = true)
 |-- detail_special_feature: string (nullable = true)
 |-- detail_hard_disk_form_factor: string (nullable = true)
 |-- detail_hard_disk_descripti

3.5.1
